In [ ]:
# 問１：READ.MEを参考に学習と推定を実施

In [36]:
# 使用してしているGPUの確認 TeslaK80<T4<P100
!nvidia-smi

Wed Jun 24 07:23:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install tensorflow==1.5.0
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 3.8MB 54.7MB/s 
     |████████████████████████████████| 512kB 50.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=719ff1a66b18e8fb5bb25cdc9e77a7ef96329a3c60654e2bf7954922aec4abca
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: 

In [3]:
import tensorflow

In [37]:
tensorflow.__version__

'1.15.0'

In [38]:
# kaggleコマンドを使うために install kaggle をする。
# kaggle コマンド、カグルデータをダウンロードするときにカグルコマンドを使います
!pip install kaggle

In [6]:
#mount を実行するとgoogleのマイドライブ情報を参照出来るようになる
# ログに出てくるリンクを進めて出てくるワンアクションID的なIDで通過させる

from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [39]:
mkdir .kaggle

In [40]:
ls -a

annotation.txt  .gitignore           model/   predict.py  simpsons_dataset/
.DS_Store       .ipynb_checkpoints/  out.csv  README.md   train.py
.git/           .kaggle/             output/  save/


# New Section

In [41]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

import json

token = {"username":"shuji0001","key":"d13387a1e8319ff69a7c00c83da993a9"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [42]:
!chmod 600 /content/.kaggle/kaggle.json

In [52]:
pwd

'/content'

In [53]:
ls -a .kaggle

./  ../  kaggle.json


In [54]:
mkdir　/root/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [55]:
# ルートに保存してあるkaggle.jsonをcontentにコピー
!cp -R /content/.kaggle/kaggle.json /root/.kaggle/

In [56]:
!ls -a /root/.kaggle/

.  ..  kaggle.json


In [57]:
# kaggle APIコマンドが使えるかどうかを確認


!kaggle competitions list

ref                                               deadline             category            reward  teamCount  userHasEntered  
------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
tpu-getting-started                               2030-06-03 23:59:00  Getting Started      Kudos         61           False  
digit-recognizer                                  2030-01-01 00:00:00  Getting Started  Knowledge       3461           False  
titanic                                           2030-01-01 00:00:00  Getting Started  Knowledge      26208            True  
house-prices-advanced-regression-techniques       2030-01-01 00:00:00  Getting Started  Knowledge       5890            True  
connectx                                          2030-01-01 00:00:00  Getting Started  Knowledge        590           False  
nlp-getting-started                               2030-01-01 00:00:00  Getting Started      Kudos       1856   

In [58]:
# カレントディレクトリの変更

import os

os.chdir('/content/drive/My Drive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive


In [ ]:
# simpsonデータセットのAPIコマンドでデータセットをダウンロードする
# APIコマンドはこちらのサイトを参照。https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

# !kaggle datasets download -d alexattia/the-simpsons-characters-dataset

the-simpsons-characters-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# シンプソンデータを解凍して、Simpsonsに入れる
# 時間かかる。。15分程度
# ドライブには保存していあるから回答しないで[N]を選択する
# !unzip /content/drive/'My Drive'/the-simpsons-characters-dataset.zip  -d Simpsons

Archive:  /content/drive/My Drive/the-simpsons-characters-dataset.zip
replace Simpsons/annotation.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [19]:
# cd ../

In [22]:
# FasterRCnnモデルにシンプソンデータを使用したテンプレをクローン→クローンしたらローカルにあるfasterRCNNの内容をreadmeを読んでupdateする
# google ドライブに既にクローンがあるなら不要
# !git clone https://github.com/duckrabbits/ObjectDetection.git

Cloning into 'ObjectDetection'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 46 (delta 3), reused 6 (delta 1), pack-reused 28
Unpacking objects: 100% (46/46), done.


In [ ]:
# https://github.com/duckrabbits/ObjectDetection/tree/master
# この後、上述のgithubのreadmeに沿って、simpsons_datasetフォルダをプロジェクトフォルダ配下（ObjectDetection）に格納しましょう。
# testデータなども同様に準備してみよう。

In [59]:
cd ObjectDetection

/content/drive/My Drive/ObjectDetection


In [60]:
!pip install keras==2.1.5

In [63]:
# ####注意#####時間が1時間以上掛かるので実行はちゃんと考えてから実施する。なぜかは不明。。
# 学習
# 時間が掛かる？？動かない。。なぜ？？→時間は掛からない！何かがおかしい！
# シンプソンデータは手動で ObjectDetction 配下に移動させよう！→既にドライブに移動済み
# model>config K.image -> K.common.image に変更。
# model>FixedBatchNormalization.py shape=shape
# model>FixedBatchNormalization.py line73 axis=0削除
!python train.py -p annotation.txt

Using TensorFlow backend.
Parsing annotation files
Training images per class (19 classes) :
{'abraham_grampa_simpson': 687,
 'apu_nahasapeemapetilon': 206,
 'bart_simpson': 650,
 'bg': 0,
 'charles_montgomery_burns': 650,
 'chief_wiggum': 209,
 'comic_book_guy': 208,
 'edna_krabappel': 212,
 'homer_simpson': 718,
 'kent_brockman': 213,
 'krusty_the_clown': 429,
 'lisa_simpson': 756,
 'marge_simpson': 629,
 'milhouse_van_houten': 210,
 'moe_szyslak': 403,
 'ned_flanders': 675,
 'nelson_muntz': 219,
 'principal_skinner': 614,
 'sideshow_bob': 201}
-------------------------------
path to config file : ./save/train_20200624-074329_config.pickle
-------------------------------









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLaye

In [64]:
# 推定
# path指定のMy Driveはスペースが入っているのでシングルコーテーションで挟む
# python predict.py -i [推定したい画像を入れたディレクトリのパス] -c [学習時に保存された設定ファイルのパス]
# 推定したい画像のディレクトリのパスは シンプソンデータが入ってるディレクトリのパス、右クリックで参照可能
# 学習時に保存された設定ファイルのパスは、train.py のログの最後に出力されたファイルのパスを設定
# 学習が足りない（10epoch程度）と推定しても、学習不足で推定結果を出せない
!python predict.py -i /content/drive/'My Drive'/ObjectDetection/simpsons_dataset -c /content/drive/'My Drive'/ObjectDetection/save/train_20200624-074329_config.pickle

Using TensorFlow backend.










2020-06-24 07:46:26.857211: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-24 07:46:26.862094: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2020-06-24 07:46:26.862305: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xa406000 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-24 07:46:26.862334: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-06-24 07:46:26.864435: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-24 07:46:26.970525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

In [ ]:
# 問２　train.pyで呼び出しているファイルを見てみる。少し関連するファイルの範囲が広い。
"""
・RPNを実現している行はどこか？
69行目でrpnの指定を受けている？fasterrcnn.py
・RoIプーリングを実現している行はどこか？
118~125。。。かな？roipoolingconv.py,roi:helpers.py
"""

In [ ]:
# 問３ 進め方
# https://github.com/qqwweee/keras-yolo3 のクイックスタートをみて実装する。
# 下記の順番は順不同だけどあまり気にしない。

# Download YOLOv3 weights from YOLO website.→YOLOウェブサイトからYOLOv3ウェイトをダウンロードします。→
# ! wget https://pjreddie.com/media/files/yolov3.weights　これを実行する

# Convert the Darknet YOLO model to a Keras model.→Darknet YOLOモデルをKerasモデルに変換します。→
# ! git clone https://github.com/qqwweee/keras-yolo3 keras_yolo3変換モデルが存在するのでこれをクローンする。
# ! python convert.py yolov3.cfg /content/yolov3.weights model_data/yolo.h5 →yolo3の中に入って変換モデルを使う

# Run YOLO detection.→YOLO検出を実行します。→yolo_video.pyで検出する
# python yolo_video.py --image を使う
# ただ、yolo_video.pyをダウンロードして、書き換える必要がある。内容は別セルに記述。

In [21]:
pwd

'/content/drive/My Drive/ObjectDetection'

In [22]:
cd ..

/content/drive/My Drive


In [ ]:
# 時間かかる。。15分程度
# もうドライブに保存してあるから実施しないでも良い。
# ! wget https://pjreddie.com/media/files/yolov3.weights

--2020-03-18 11:13:41--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   459KB/s    in 13m 37s 

2020-03-18 11:27:19 (296 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
# あるので実施しないでも良い。
# Kerasモデルに変換するモデル
# ! git clone https://github.com/qqwweee/keras-yolo3

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 1.09 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [23]:
cd keras-yolo3

/content/drive/My Drive/keras-yolo3


In [ ]:
# yolo3をkeras に書き換える
# 既に実施してあるので不要
#! python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5　# サンプル 動かない
# ! python convert.py yolov3.cfg ./../yolov3.weights model_data/yolo.h5 # サンプル２　動く
# ! python convert.py yolov3.cfg /content/drive/'My Drive'/yolov3.weights model_data/yolo.h5 # サンプル３これを使う

Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)




2020-03-19 05:48:35.479087: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-19 05:48:35.481661: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x13d2a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-19 05:48:35.481701: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-19 05:48:35.487400: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-19 05:48:35.647750: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node

In [24]:
# 以下実行前に削除するものがある。修正するものがある。
# python yolo_video.py [OPTIONS...] --image, for image detection mode, OR
# yoro_video.py を修正　→ r_image.show()の下の行に次の行を挿入 r_image.save('output.jpg') -> 実行後はoutput.jpgを確認する
# 実行すると何度もinput-imgを求められる。。終了方法が分からない。→プログラムは自分で終了させる。input_imageを入れるとoutput.jpgが生成されるのでそれで終了。
! python yolo_video.py --image

Using TensorFlow backend.
Image detection mode
 Ignoring remaining command line arguments: ./path2your_video,



2020-06-24 04:34:12.641764: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-24 04:34:12.647043: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2020-06-24 04:34:12.647332: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2ca4bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-24 04:34:12.647385: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-06-24 04:34:12.652377: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-24 04:34:12.824805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had neg

KeyboardInterrupt: ignored

In [ ]:
# cd ../drive/'My Drive'

In [ ]:
# !mv /content/drive/'My Drive'/Simpsons/simpsons_dataset /content/keras-yolo3
# !cp -rp Simpsons/simpsons_dataset /content/keras-yolo3

In [ ]:
# 手動で シンプソンデータを keras-yolo3 に移動させても良い
# データをkeras-yoro3/にコピー
# mv Simpsons/simpsons_dataset keras-yoro3/
# 

In [25]:
pwd

'/content/drive/My Drive/keras-yolo3'

In [33]:
# 学習後にmodel_data/yolo_weights.h5が作成される。
!python convert.py -w yolov3.cfg /content/drive/'My Drive'/yolov3.weights model_data/yolo_weights.h5

Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-06-23 06:52:13.962711: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-23 06:52:13.989891: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-23 06:52:13.990567: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P4 major: 6 minor: 1 memoryClockRate(GHz): 1.1135
pciBusID: 0000:00:04.0
2020-06-23 06:52:13.990850: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
20

In [ ]:
# 問４はアノテーションファイルとクラスィーズファイルを作成する（アンカーファイルは変更不要、、なぜ？）

# トレーニングデータであるシンプソンデータのアノテーションファイルを
# GitHubのトレーニングの形式に書き換える必要があるよ。
# train.py に書かれている annotation_path='train.txt'のtrain.txtを用意する。最後はラベルにする（グランパは11）
# train.txt はシンプソンデータのアノテーションファイルの内容で形式はGitHubの通りに揃える（vscodeで全置換）

# classes_path のvoc_classes.txtは初めはcocoデータの内容（自転車とか）に
# なっているのでこれをシンプソンデータのクラス名に変更しよう（エクセルで重複削除）


In [30]:
# 問５：学習が出来る事を確認する
# 問４の対応が終わったら通る。。はず！！通った！！

# K.control_flow_ops.while_loop to→tf.while_loop. に変更
! python train.py

Using TensorFlow backend.








2020-06-24 05:02:04.116650: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-24 05:02:04.121127: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2020-06-24 05:02:04.121311: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1f6abc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-24 05:02:04.121337: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-06-24 05:02:04.123066: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-24 05:02:04.224124: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [44]:
# AttributeError: module 'keras.backend' has no attribute 'control_flow_ops'
# というエラーがでた。調査をすると、このURLそれっぽいことが書いてある。
# https://github.com/qqwweee/keras-yolo3/issues/544
# change K.control_flow_ops.while_loop to tf.while_loop. エラーメッセージの前の2行にもmodel.pyの394行目が怪しい
# とあるのでそこを修正する。
# またエラー発生。　No such file or directory: './characters/charles_montgomery_burns/pic_0269.jpg'
# 参照先のファイルが無い、というようなエラー。。。どこを修正すれば良いのか不明。。
# →characters/***/XXX.jpgはシンプソンデータを読みに行こうとしているので sinmpson_datasetをコピーして、
# ディレクトリ名を「character」に変更すると読める。
# またまたエラー「ValueError: invalid literal for int() with base 10: 'bart_simpson'」
# →yoloのreadmeをよく読む、train.txtの末尾のクラス名をクラス番号に変更する。
# またまたエラー「FileNotFoundError: [Errno 2] No such file or directory: './characters2/sideshow_bob/pic_0807.jpg'」
# →characters2というディレクトリが必要なの？キリがない気がする。。train.txtを見ると
# ./characters2/sideshow_bob/pic_0690.jpgというのがあるのでcharacters2は消す。。やっと通った！！！！

In [ ]:
pwd

'/content/drive/My Drive/keras-yolo3'